# NLP Assignment: Language Modelling with Poetry (deadline **20.5.2025**)

This assignment will check your basic NLP understanding through the fundamental NLP task of **language modelling**.

You will reiterate on the task with techniques ranging from simple n-gram counting to embeddings and deep learning.


You will work with the same short poetry texts that should be very familiar to you by now:

In [2]:
!wget -nc https://raw.githubusercontent.com/GustikS/smu-nlp/master/robert_frost.txt
!wget -nc https://raw.githubusercontent.com/GustikS/smu-nlp/master/edgar_allan_poe.txt

File ‘robert_frost.txt’ already there; not retrieving.

File ‘edgar_allan_poe.txt’ already there; not retrieving.



Unless stated otherwise (Ex. 4,5,6), work just with the Robert Frost file for simplicity (Ex. 1,2,3,7,8)

In [3]:
!head robert_frost.txt

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there


In [5]:
import pandas as pd
import numpy as np
import nltk
from sklearn.decomposition import TruncatedSVD
import plotly.express as px
from nltk import word_tokenize

nltk.download('punkt_tab')

np.random.seed(135671)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/tomkys144/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


#### **Excercise 1**: Markov Language Model (**2 points**)

1. Create a 1st order Markov (bi-gram) language model
  - work with matrix representation of the model
    - i.e. not dictionaries as we did in the tutorial!
    - hence you'll need to assign indices to the words, too
      - include an extra \<UNK\> token for unseen words
  - correctly handle beginnings and ends of sentences
    - sentence = line (skip empty lines)
  - lower case and properly tokenize your sentences
    - but skip all other text preprocessing

In [8]:
def extractSentences(filepath):
  with open(filepath) as f:
    lines = f.readlines()

  sentences = []
  for line in lines:
    if line.strip() == '':
      continue
    sentences.append(line.strip().lower().split())

  return sentences

def makeVocabulary(sentences):
  vocabulary = {'<UNK>' : 0}
  invVocabulary = {0: '<UNK>'}

  wordIndex = 1;
  for sentence in sentences:
    for word in sentence:
      if word not in vocabulary:
        vocabulary[word] = wordIndex;
        invVocabulary[wordIndex] = word;
        wordIndex += 1;

  vocabulary['<START>'] = wordIndex;
  invVocabulary[wordIndex] = '<START>'

  vocabulary['<END>'] = wordIndex + 1;
  invVocabulary[wordIndex + 1] = '<END>'

  return vocabulary, invVocabulary

def makeTransitionMatrix(vocabulary, sentences):
  transitionMatrix = np.zeros((len(vocabulary), len(vocabulary)))

  sentences_idx = []
  for sentence in sentences:
    sentence_idx = [vocabulary.get(word, vocabulary['<UNK>']) for word in sentence]
    sentence_idx = [vocabulary['<START>']] + sentence_idx + [vocabulary['<END>']]
    sentences_idx.append(sentence_idx)

  for sentence in sentences_idx:
    for i in range(0, len(sentence) - 1):
      transitionMatrix[sentence[i], sentence[i+1]] += 1

  row_sums = transitionMatrix.sum(axis=1, keepdims=True)
  transitionMatrix = np.divide(transitionMatrix, row_sums, out=np.zeros_like(transitionMatrix), where=row_sums!=0)

  return transitionMatrix

def sampleWord(probList, invVocabulary):
  p0 = np.random.random()

  cum = 0
  it = np.nditer(probList, flags=['f_index'])
  idx = 0
  for p in it:
    cum += p
    if p0 < cum:
      idx = it.index
      return invVocabulary[idx], idx


def generate1(numLines, transitionMatrix, vocabulary, invVocabulary):
  for _ in range(numLines):
    sentence = []
    w0 = '<START>'
    idx0 = vocabulary[w0]
    w1 = ''
    idx1 = 0

    while True:
      w1, idx1 = sampleWord(transitionMatrix[idx0], invVocabulary)
      if w1 == '<END>':
        break
      sentence.append(w1)
      w0 = w1
      idx0 = idx1

    print(' '.join(sentence))

In [9]:
sentences = extractSentences('robert_frost.txt')
vocabulary, invVocabulary = makeVocabulary(sentences)
transitionMatrix = makeTransitionMatrix(vocabulary, sentences)

generate1(6, transitionMatrix, vocabulary, invVocabulary)

up his lantern, saying, "ile's
might just see if all four of its face.
with oil of the proof.'
or vase or some keeping still said to keep.
and miles could have to get nails.' i don't they?) by the crater's verge
somewhere with him by the way he's awful!


#### **Excercise 2**: Probability + Smoothing (**2 points**)
1. write a function to obtain probability of a given sentence with your model
    - include the beginning and end mark of the sentence as well
    - test also some other sentences and assure the probability makes sense
      - write down your reasoning!
2. incorporate the add-1 (Laplace) smoothing to account for unseen bi-grams
    - you should have your \<UNK\> for unseen unigrams already
    - calculate prob. of "*the young folk held some hope out to each other.*"

In [10]:
def makeTransitionMatrixSmoothed(vocabulary, sentences, smoothing = 1.0):
  transitionMatrix = np.zeros((len(vocabulary), len(vocabulary)))

  sentences_idx = []
  for sentence in sentences:
    sentence_idx = [vocabulary.get(word, vocabulary['<UNK>']) for word in sentence]
    sentence_idx = [vocabulary['<START>']] + sentence_idx + [vocabulary['<END>']]
    sentences_idx.append(sentence_idx)

  for sentence in sentences_idx:
    for i in range(0, len(sentence) - 1):
      transitionMatrix[sentence[i], sentence[i+1]] += 1

  transitionMatrix += smoothing

  row_sums = transitionMatrix.sum(axis=1, keepdims=True)
  transitionMatrix = np.divide(transitionMatrix, row_sums, out=np.zeros_like(transitionMatrix), where=row_sums!=0)

  return transitionMatrix

def sentenceProb(sentence, vocabulary, transitionMatrix, verbose = False):
  words = sentence.strip().lower().split()
  words = ['<START>'] + words + ['<END>']

  p = 1.0
  for idx in range(len(words) - 1):
    idx0 = vocabulary.get(words[idx], vocabulary['<UNK>'])
    idx1 = vocabulary.get(words[idx+1], vocabulary['<UNK>'])

    p01 = transitionMatrix[idx0][idx1]
    if verbose:
      print(words[idx], '->', words[idx+1], ': ', p01)

    p *= p01
  return p

In [11]:
sentence = "the young folk held some hope out to each other."

transitionMatrixSmoothed = makeTransitionMatrixSmoothed(vocabulary, sentences)

p = sentenceProb(sentence, vocabulary, transitionMatrixSmoothed, True)
print('p: ', np.format_float_scientific(p, precision=4))

<START> -> the :  0.01713776986423325
the -> young :  0.0005561735261401557
young -> folk :  0.0006533812479581836
folk -> held :  0.0006538084341288003
held -> some :  0.0006533812479581836
some -> hope :  0.0006470397929472663
hope -> out :  0.00065359477124183
out -> to :  0.0016129032258064516
to -> each :  0.000892325996430696
each -> other. :  0.0013050570962479609
other. -> <END> :  0.00065359477124183
p:  1.3812e-33




*The probability close to 0 is expected thanks to sparse transition matrix with lot's of small probabilities*



#### **Excercise 3**: Perplexity (**1 points**)
1. write a function fo calculate perplexity of your smoothed model on a given sentence
  - including its beginning and ending
2. find the sentence(s) from the corpus with minimum and maximum perplexity

In [12]:
def sentencePerplexity(sentence, transitionMatrix):
  words = sentence.strip().lower().split()
  n = len(words)

  words = ['<START>'] + words + ['<END>']

  p = 1

  for idx in range(len(words) - 1):
    idx0 = vocabulary.get(words[idx], vocabulary['<UNK>'])
    idx1 = vocabulary.get(words[idx+1], vocabulary['<UNK>'])

    p01 = transitionMatrix[idx0][idx1]

    p *= 1/p01

  pp = p ** 1/n
  return pp

def findMinMaxPP(sentences, transitionMatrix):
  minPP = np.inf
  maxPP = 0

  minSentence = ""
  maxSentence = ""

  for sentence in sentences:
    sentenceFull = " ".join(sentence)
    pp = sentencePerplexity(sentenceFull, transitionMatrix)

    if pp > maxPP:
      maxPP = pp
      maxSentence = sentenceFull

    if pp < minPP:
      minPP = pp
      minSentence = sentenceFull

  return minSentence, maxSentence, minPP, maxPP

In [13]:
sentence = "the young folk held some hope out to each other."
pp = sentencePerplexity(sentence, transitionMatrixSmoothed)
print('pp: ', np.format_float_scientific(pp, precision=4))

minSentence, maxSentence, minPP, maxPP = findMinMaxPP(sentences, transitionMatrixSmoothed)
print("MIN: ", minSentence, ' | pp: ', np.format_float_scientific(minPP, precision=4))
print("MAX: ", maxSentence, ' | pp: ', np.format_float_scientific(maxPP, precision=4))

pp:  7.2401e+31
MIN:  it.  | pp:  4.3189e+05
MAX:  mother: yes, we could too, son. tell the truth for once.  | pp:  2.0785e+36


#### **Excercise 4:**  Markov classifier (**4 points**)
Implement your own probabilistic classifier based on your bi-gram language model. That is:
  1. given some classes of sentences, train a separate language model for each class respectively
  2. then classify a given sentence (=sample) based on maximum a-posteriori probability (MAP)
    - i.e. don't forget about the class priors, too!
    - use log probabilities!
    - make sure your smoothing treats all the classes equally!
      - ...think about what happens to the UNK token
  3. evaluate on a task of recognizing sentences from the 2 different poets (Frost vs. Poe)
    - split the sentences (=samples) from each poet into train-test in advance!
      - fixed train-test split 70:30
        - do not shuffle sentences
      - skip empty lines (of course)
 	- report all accuracy values + a confusion matrix

*Note that this is very similar to our previous classification with Naive Bayes, but with bi-grams instead of unigrams.*

In [14]:
def splitData(sentences, perc = 0.70):
  wholeLen = len(sentences)
  trainLen = int(np.floor(perc * wholeLen))

  trainSentences = sentences[ : trainLen]
  testSentences = sentences[trainLen : ]

  return trainSentences, testSentences

def trainModel(sentences0, sentences1, vocabulary):
  matrix0 = makeTransitionMatrixSmoothed(vocabulary, sentences0)
  matrix1 = makeTransitionMatrixSmoothed(vocabulary, sentences1)

  totalLen = len(sentences0) + len(sentences1)
  prior0 = len(sentences0) / totalLen
  prior1 = len(sentences1) / totalLen
  logPrior0 = np.log(prior0)
  logPrior1 = np.log(prior1)

  return {'m0': matrix0, 'm1': matrix1, 'p0': logPrior0, 'p1':  logPrior1}

def classify(sentence, vocabulary, model):
  p0 = sentenceProb(sentence, vocabulary, model['m0'])
  logP0 = np.log(p0) + model['p0']

  p1 = sentenceProb(sentence, vocabulary, model['m1'])
  logP1 = np.log(p1) + model['p1']

  return 0 if logP0 > logP1 else 1

def evaluateModel(sentences0, sentences1, vocabulary, model):
  labelsTrue = []
  labelsPred = []

  for sentence in sentences0:
    labelsTrue.append(0)
    labelsPred.append(classify(" ".join(sentence), vocabulary, model))

  for sentence in sentences1:
    labelsTrue.append(1)
    labelsPred.append(classify(" ".join(sentence), vocabulary, model))

  totalLen = len(labelsTrue)

  correct = [labelsTrue[i] == labelsPred[i] for i in range(totalLen)]
  acc = sum(correct) / len(labelsTrue)

  T0 = sum([labelsTrue[i] == labelsPred[i] and labelsPred[i] == 0 for i in range(totalLen)])
  F0 = sum([labelsTrue[i] != labelsPred[i] and labelsPred[i] == 0 for i in range(totalLen)])
  T1 = sum([labelsTrue[i] == labelsPred[i] and labelsPred[i] == 1 for i in range(totalLen)])
  F1 = sum([labelsTrue[i] != labelsPred[i] and labelsPred[i] == 1 for i in range(totalLen)])

  cm = [[T0, F1], [F0, T1]]

  return acc, cm

In [15]:
sentences_frost = extractSentences('robert_frost.txt')
sentences_poe = extractSentences('edgar_allan_poe.txt')

train_frost, test_frost = splitData(sentences_frost)
train_poe, test_poe = splitData(sentences_poe)

vocabulary, _ = makeVocabulary(train_frost + train_poe)

model = trainModel(train_frost, train_poe, vocabulary)

acc, cm = evaluateModel(test_frost, test_poe, vocabulary, model)

print("Accuracy: ", acc)
print("Confusion Matrix:")
print("      | FROST | POE")
print("FROST | ", cm[0][0], " | ", cm[0][1])
print("POE   | ", cm[1][0], " | ", cm[1][1])

Accuracy:  0.8469860896445132
Confusion Matrix:
      | FROST | POE
FROST |  415  |  16
POE   |  83  |  133


#### **Excercise 5**: PPMI word-word cooccurence (**1 points**)
1. Create a word-word co-occurence matrix from all the text of both the poets altogether
  - use a sliding window of size 5 (2 left + 2 right context words)
    - remember that you can reuse existing solutions...
2. Post-process the matrix with PPMI

In [16]:
def cooccurance(text, contextSZ):
  winsize = (contextSZ - 1) // 2;
  cooccuranceMatrix = {}
  vocabulary = set()

  tokens = word_tokenize(text)
  tokens

  for token in tokens:
    vocabulary.add(token)

  for i in range(winsize, len(tokens) - winsize):
    token = tokens[i]
    context = tokens[(i - winsize): i] + tokens[(i + 1) : (i + 1 + winsize)]
    for t in context:
      key = tuple(sorted([t, token]))
      cooccuranceMatrix[key] = cooccuranceMatrix.get(key, 0) + 1

  vocab = sorted(vocabulary)
  df = pd.DataFrame(data=np.zeros((len(vocabulary), len(vocabulary)), dtype=np.int16), index=list(vocabulary), columns=list(vocabulary))
  for key, value in cooccuranceMatrix.items():
    df.at[key[0], key[1]] = value
    df.at[key[1], key[0]] = value

  return df

def ppmi(df, positive = True):
  colSums = df.sum(axis=0)
  rowSums = df.sum(axis=1)

  total = colSums.sum()

  expected = np.outer(rowSums, colSums) / total

  df = np.log(df / expected)

  df[np.isinf(df)] = 0.0

  if positive:
    df[df < 0] = 0.0

  return df

In [17]:
text_frost = " ".join([" ".join(sentence) for sentence in sentences_frost])
text_poe = " ".join([" ".join(sentence) for sentence in sentences_poe])

text = text_frost + text_poe

cooccuranceDF = cooccurance(text, 5)
cooccuranceDF

,farm,mountains-,hardest,inscription,acquainted,removed,giddy,did,up,nor,...,'in,lived,transfixed,shook,shudder,in-,idea,girl,flying,ahead
farm,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
mountains-,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hardest,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
inscription,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
acquainted,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
in-,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
idea,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
girl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
flying,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
ppmiDF = ppmi(cooccuranceDF)
ppmiDF

/home/tomkys144/Documents/CTU/b242/SMU/HW/03/venv/lib64/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


,farm,mountains-,hardest,inscription,acquainted,removed,giddy,did,up,nor,...,'in,lived,transfixed,shook,shudder,in-,idea,girl,flying,ahead
farm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mountains-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
hardest,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
inscription,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
acquainted,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
in-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
idea,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
girl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
flying,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### **Excercise 6**: Word embeddings (**1 points**)
1. Extract 8-dimensional word embeddings from your PPMI matrix
  - using the truncated SVD matrix decomposition
2. Plot them in 2D with word labels

In [20]:
svd = TruncatedSVD(n_components=10)

Z = svd.fit_transform(ppmiDF.values)

fig = px.scatter(x=Z[:,0], y=Z[:,1], text=ppmiDF.columns, size_max=60)
fig.update_traces(textposition='top center')
fig.show()

#### **Excercise 7:** LSTM Language Model (**3 points**)
1. Formulate a proper dataset for language modelling on longer sequences from the text
  - beyond the n-gram scope, use 10 consecutive words
2. Create a suitable LSTM-based language model
3. Initialize the embedding layer of your model with your "pretrained" SVD embeddings
4. Train the model in a suitable fashion

In [21]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [22]:
class PoetryDataset(Dataset):
  def __init__(self, sentences, vocab, contextSZ=10):
    super().__init__()
    self.vocab = vocab
    self.data = []
    for sentence in sentences:
      words = ['<START>'] * 9 + sentence + ['<END>']
      idxs = [vocab.get(word, vocab['<UNK>']) for word in words]
      for i in range(len(idxs) - contextSZ):
        context = idxs[i:i+contextSZ]
        target = idxs[i+contextSZ]
        self.data.append((context, target))

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    context, target = self.data[index]
    return torch.tensor(context), torch.tensor(target)

class LSTMModel(nn.Module):
  def __init__(self, vocabSZ, embeddingDim = 16, hiddenDim = 16, numLayers = 2, pretrainedEmbeddings = None):
    super().__init__()
    self.embedding = nn.Embedding(vocabSZ, embeddingDim)
    if pretrainedEmbeddings is not None:
      self.embedding.weight.data.copy_(torch.from_numpy(pretrainedEmbeddings))

    self.lstm = nn.LSTM(embeddingDim, hiddenDim, numLayers, batch_first=True)
    self.fc = nn.Linear(hiddenDim, vocabSZ)

  def forward(self, x):
    emb = self.embedding(x)
    output, _ = self.lstm(emb)
    logits = self.fc(output[:, -1, :])
    return logits

def trainLSTM(model, dataloader, epochs=5, lr=0.01):
  device = torch.device("cpu")
  model = model.to(device)
  lossF = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)

  model.train()
  for epoch in range(epochs):
    totalLoss = 0
    for xBatch, yBatch in dataloader:
      xBatch, yBatch = xBatch.to(device), yBatch.to(device)

      optimizer.zero_grad()
      logits = model(xBatch)
      loss = lossF(logits, yBatch)
      loss.backward()
      optimizer.step()

      totalLoss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {totalLoss:.4f}")

def generateLSTM(numLines, model, vocabulary, invVocabulary):
  for _ in range(numLines):
    model.eval()

    w0 = vocabulary.get('<START>', vocabulary['<UNK>'])
    wSeq = [w0] * 10

    sentence = []

    while True:
      x = torch.tensor([wSeq]).long()
      with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits[0], dim=0).numpy()
        next_idx = np.random.choice(len(probs), p = probs)

      w1 = invVocabulary[next_idx]
      if w1 == '<END>':
        break

      sentence.append(w1)
      wSeq = wSeq[1:] + [next_idx]

    print(' '.join(sentence))

In [ ]:
trainSentences = sentences_frost + sentences_poe
vocabulary, invVocabulary = makeVocabulary(trainSentences)

dataset = PoetryDataset(trainSentences, vocabulary, 10)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

svd_embeddings = Z
if svd_embeddings.shape[0] < len(vocabulary):
    pad = np.zeros((len(vocabulary) - svd_embeddings.shape[0], svd_embeddings.shape[1]))
    svd_embeddings = np.vstack([svd_embeddings, pad])

model = LSTMModel(len(vocabulary), 10, 64,2, svd_embeddings)

trainLSTM(model, dataloader, epochs=10, lr=0.005)

 #### **Excercise 8**: Sampling (**1 point**)
1. Sample some text from your models w.r.t. to the output (next) word probability distribution
  - for both bigram and LSTM models - which is better?

2. Paste your best "poem" here:



*in an acquaintance made them really straight out of me.*

*who has been brought us*

*it's not twenty years ago as we don't go with both father had been fair*

*we've been starks, and posts*

*you want my demand upon her, and, opening out of those the bed,*

*two converging slides, the hard cellar hole,*



In [23]:
transitionMatrix = makeTransitionMatrix(vocabulary, sentences)

numLines = 6
numPoems = 4

print("##################")
print("#     BIGRAM     #")
print("##################")
for i in range(4):
  print("\n### POEM ", i+1, " ###")
  generate1(numLines, transitionMatrix, vocabulary, invVocabulary)

print("\n################")
print("#     LSTM     #")
print("################")
for i in range(4):
  print("\n### POEM ", i+1, " ###")
  generateLSTM(numLines, model, vocabulary, invVocabulary)

##################
#     BIGRAM     #
##################

### POEM  1  ###


KeyError: 3390

# **Notes**

If this seems like a lot of work, reiterate through the tutorials (which is the purpose, anyway) - we did most of this already!
  - i.e., I don't expect you to be overly creative, you can find all the pieces in the tutorial notebooks
  - but feel free to write a more efficient/clean/suitable code from scratch
  - only the libraries we used in the tutorials are allowed
  - teamwork is forbidden!
  - chatGPT is allowed

Before submitting:
  - make sure your code is runnable (in Colab)!
  - comment your code at least a bit, make it readable!
  - add printouts after every excercise
  - submit just the notebook as a single file (filename = username)


I will only evaluate *correctness* of the solutions, not quality of the models
  - i.e. no need to spent too much time with text preprocessing or hyperparameter tuning (LSTM)

# **Bonus points**
You can recover some of the potentially lost points here, but you can't reach more than 15 points in total from this assignment
  - *explicitly mark the bonus points you consider as fullfilled!*
<br/><br/>

1. (2p) Use a different (larger) textual corpus of poems
  - e.g. Corpus of Czech Verse, collected at the Institute of Czech Literature at Czech Academy of Sciences
    - available at https://github.com/versotym/corpusCzechVerse
    - just be careful about czech character encoding

2. (7p) An alternative task - create a "whisperer" (bot player) for the popular game ["Kryci jmena"](https://krycijmena.cz/) based on word embeddings
  - i.e. automatically search for words that would cover (be related/close to) subsets of the "positive" words, while trying to avoid the "negative" words
  - this will be graded based on both quality of the approach and the result

In [24]:
# Corpus of Czech Verse
!git clone https://github.com/versotym/corpusCzechVerse

Cloning into 'corpusCzechVerse'...
remote: Enumerating objects: 1328, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 1328 (delta 4), reused 0 (delta 0), pack-reused 1312 (from 1)
Receiving objects: 100% (1328/1328), 278.87 MiB | 23.16 MiB/s, done.
Resolving deltas: 100% (1302/1302), done.
Updating files: 100% (1306/1306), done.


In [27]:
import json
from os import listdir
import string

def loadBooks(numBooks, path):
  books = []
  files = [f for f in listdir(path)]
  for i, f in enumerate(sorted(files)[:numBooks]):
    # print(i, f)
    book = open(path+f)
    books.append(json.load(book))
  return books

def extractPoems(books):
  poems = []
  for book in books:
    for poem in book:
      author = poem['p_author']['name']
      title = poem['biblio']['p_title']
      body = poem['body']

      txt = []
      for text in body:
        for line in text:
          txt.append(line['text'])

      poems.append({'author': author, 'title': title, 'body': txt})
  return poems

def poems2sentences(poems, removePunc=False):
  sentences = []
  for poem in poems:
    for line in poem['body']:
      if line.strip() == '':
        continue
      if removePunc:
        line = line.translate(str.maketrans('', '', string.punctuation))
      sentences.append(line.strip().lower().split())
  return sentences

path = 'corpusCzechVerse/ccv/'
numBooks = 10 # max 1305

books = loadBooks(numBooks, path)
poems = extractPoems(books)
sentences = poems2sentences(poems, True)
vocab, invVocab = makeVocabulary(sentences)

tm = makeTransitionMatrixSmoothed(vocab, sentences)
generate1(4, tm, vocab, invVocab, 10)

text = " ".join([" ".join(sentence) for sentence in sentences])
cooccuranceDF = cooccurance(text, 5, lang='czech')
cooccuranceDF

ppmiDF = ppmi(cooccuranceDF)
ppmiDF

svd = TruncatedSVD(n_components=10)
svd_embeddings = svd.fit_transform(ppmiDF.values)
fig = px.scatter(x=svd_embeddings[:,0], y=svd_embeddings[:,1], text=ppmiDF.columns, size_max=60)
fig.update_traces(textposition='top center')
fig.show()

dataset = PoetryDataset(sentences, vocab, 10)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

if svd_embeddings.shape[0] < len(vocab):
    pad = np.zeros((len(vocab) - svd_embeddings.shape[0], svd_embeddings.shape[1]))
    svd_embeddings = np.vstack([svd_embeddings, pad])

model = LSTMModel(len(vocab), 10, 64,2, svd_embeddings)

trainLSTM(model, dataloader, epochs=10, lr=0.005)

generateLSTM(4, model, vocab, invVocab)

TypeError: generate1() takes 4 positional arguments but 5 were given